<a href="https://colab.research.google.com/github/Xueying-Ya/Tort-case-analysis/blob/main/Siamese-Neural-network/Siamese_base_line(Maxent).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pythainlp 

     |████████████████████████████████| 11.0MB 6.1MB/s 
     |████████████████████████████████| 747kB 45.6MB/s 


In [ ]:
!pip install attacut

     |████████████████████████████████| 1.3MB 5.1MB/s 
     |████████████████████████████████| 645kB 18.5MB/s 
     |████████████████████████████████| 92kB 7.0MB/s 
     |████████████████████████████████| 481kB 23.3MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=fcdd417fbdc8a95a4d8a9465428ed29a12d5eb5ce28a073db16f455f20d4f5c4
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from pythainlp import word_tokenize
from pythainlp.tokenize import attacut
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score, precision_score, recall_score,confusion_matrix
import json

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction import DictVectorizer
import sklearn.model_selection

#Feature engineering functions

In [ ]:
def word_overlap(s1,s2):
    overlap_token = set(s1) & set(s2)
    return Counter(overlap_token)

In [ ]:
word_overlap(word_tokenize('สวัสดีปีใหม่ท้องฟ้าแจ่มใสมากๆเลย',engine = "attacut") , word_tokenize('สวัสดีอยู่บ้านจ้า',engine = "attacut"))

Counter({'สวัสดี': 1})

In [ ]:
def word_cross_product(s1,s2,feature_list):
    hyponyms = []
    hypernyms = []
    result = list(set([(token1 + '_' +token2) for token1 in s1 for token2 in s2]))
    if 'wordnet_feature_po' in feature_list:
        hyponyms = wordnet_feature('hyponyms',result)
        
    if 'wordnet_feature_per' in feature_list:
        hypernyms = wordnet_feature('hypernyms',result)
    result.extend(hyponyms + hypernyms)
    return Counter(result)

In [ ]:
from pythainlp.corpus import wordnet
def wordnet_feature(method,product_list):
    pairs = set()
    for key in list(product_list):
        w1 = key.split('_')[0]
        w2 = key.split('_')[1]
        if wordnet.synsets(w1) != [] and wordnet.synsets(w2) != []:
            hyps = sum([ss.lemma_names('tha') for ss in getattr(wordnet.synsets(w1)[0],method)()],[])
            syns = wordnet.synsets(w2)[0].lemma_names('tha')
            if set(hyps) & set(syns):
                pairs.add(w1 + '_' + w2)
            if len(w1) != len(w2) and (w1 in w2 or w2 in w1): #Ex กษัตริย์ กับ กษัตริย์สเปน
                pairs.add(w1 + '_' + w2)
    return list(pairs)

[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
word_cross_product(word_tokenize('สวัสดีปีใหม่ท้องฟ้าแจ่มใสมากๆเลย',engine = "attacut") , word_tokenize('สวัสดีอยู่บ้านจ้า',engine = "attacut"),['word_overlap' , 'word_cross_product' , 'wordnet_feature_po', 'wordnet_feature_per' ] )

Counter({'ท้อง_จ้า': 1,
         'ท้อง_บ้าน': 1,
         'ท้อง_สวัสดี': 1,
         'ท้อง_อยู่': 1,
         'ปี_จ้า': 1,
         'ปี_บ้าน': 1,
         'ปี_สวัสดี': 1,
         'ปี_อยู่': 1,
         'ฟ้า_จ้า': 1,
         'ฟ้า_บ้าน': 1,
         'ฟ้า_สวัสดี': 1,
         'ฟ้า_อยู่': 1,
         'มาก_จ้า': 1,
         'มาก_บ้าน': 1,
         'มาก_สวัสดี': 1,
         'มาก_อยู่': 1,
         'สวัสดี_จ้า': 1,
         'สวัสดี_บ้าน': 1,
         'สวัสดี_สวัสดี': 1,
         'สวัสดี_อยู่': 1,
         'เลย_จ้า': 1,
         'เลย_บ้าน': 1,
         'เลย_สวัสดี': 1,
         'เลย_อยู่': 1,
         'แจ่มใส_จ้า': 1,
         'แจ่มใส_บ้าน': 1,
         'แจ่มใส_สวัสดี': 1,
         'แจ่มใส_อยู่': 1,
         'ใหม่_จ้า': 1,
         'ใหม่_บ้าน': 1,
         'ใหม่_สวัสดี': 1,
         'ใหม่_อยู่': 1,
         'ๆ_จ้า': 1,
         'ๆ_บ้าน': 1,
         'ๆ_สวัสดี': 1,
         'ๆ_อยู่': 1})

In [ ]:
wordnet.synsets('ราชา')[0].lemma_names('tha')

['ราชา', 'พระราชา', 'กษัตริย์', 'พระมหากษัตริย์']

In [ ]:
wordnet.synsets('ราชา')[0].hyponyms()

[Synset('king_of_england.n.01'),
 Synset('king_of_france.n.01'),
 Synset('king_of_the_germans.n.01')]

In [ ]:
getattr(wordnet.synsets('ราชา')[0],'hyponyms')()

[Synset('king_of_england.n.01'),
 Synset('king_of_france.n.01'),
 Synset('king_of_the_germans.n.01')]

In [ ]:
wordnet_feature('hyponyms', word_cross_product(['ราชา'],['พระราชา'],['word_overlap' , 'word_cross_product' , 'wordnet_feature_po', 'wordnet_feature_per' ]))

['ราชา_พระราชา']

In [ ]:
for ss in getattr(wordnet.synsets('ราชา')[0],'hyponyms')():
        word = str(ss)[7:len(str(ss))-1]
        print(ss.lemma_names('tha'))
       

['กษัตริย์อังกฤษ']
['กษัตริย์ฝรั่งเศส']
['กษัตริย์เยอรมัน']


In [ ]:
print(wordnet.synset('king_of_france.n.01').lemma_names('tha'))  

['กษัตริย์ฝรั่งเศส']


# Data preparation

In [ ]:
def load_fasttext_fast(word_index, max_words, embed_size,file_name = "../input/word-vec-thai/cc.th.300.vec"):
    EMBEDDING_FILE = file_name
    emb_mean, emb_std = -0.0033470048, 0.109855264

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8") as f:       
        for line in f:
            if len(line) <= 100:
                continue
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= max_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
import string
def process_input(num_words, X_train, X_test, X_dev):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<UNK>') 
    tokenizer.fit_on_texts(X_train)

    #convert text data to numerical indexes
    train_seqs = tokenizer.texts_to_sequences(X_train)
    dev_seqs = tokenizer.texts_to_sequences(X_dev)
    test_seqs = tokenizer.texts_to_sequences(X_test)

    max_sequnce_len = max([len(x) for x in train_seqs])

    train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, maxlen=max_sequnce_len, padding="post")
    test_seqs=tf.keras.preprocessing.sequence.pad_sequences(test_seqs, maxlen=max_sequnce_len, padding="post")
    dev_seqs=tf.keras.preprocessing.sequence.pad_sequences(dev_seqs, maxlen=max_sequnce_len, padding="post")

    return  train_seqs, test_seqs, dev_seqs, max_sequnce_len, tokenizer

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/legal_tech_2021/processed_torts20210321.pkl')
df.head()

,legal_encoded,case_id,plaintiff_token,defendant_token,legal_name,legal_section,legal_content_token
0,687,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายแพ่งและพาณิชย์,420,"[ผู้, ใด, จงใจ, หรือ, ประมาท, เลินเล่อทำ, ต่อ,..."
1,251,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
2,257,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,60,"[คู่, ความ, ฝ่าย, ใด, ฝ่าย, หนึ่ง, หรือ, ผู้, ..."
3,259,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,62,"[ทนายความ, ซึ่ง, คู่, ความ, ได้, ตั้ง, แต่ง, น..."
4,135,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,142,"[คำ, พิพากษา, หรือ, คำ, สั่ง, ของ, ศาล, ที่, ช..."


In [ ]:
idx = {251 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 55',
692 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 425',
704 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 438',
154 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 172',
135 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 142',
715 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 448',
688 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 421',
202 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 249',
100 : 'ประมวลกฎหมายวิธีพิจารณาความอาญา 46',
417 :  'ประมวลกฎหมายแพ่งและพาณิชย์ 1336'}

In [ ]:
sorted_idx = list(idx.keys())

In [ ]:
def get_data(file = '../input/legal-01042021/filt_df.pickle'):
    X = [] 
    Y = []
    legal_encoded_seq = []
    filt_df = pd.read_pickle(file)
    for i,row in filt_df.iterrows():
        x = row['plaintiff_token']
        x2 = row['legal_content_token']
        X.append([x, x2])
        Y.append(row['label'])
        legal_encoded_seq.append(row['legal_encoded'])
    X = np.array(X ,dtype=object)
    Y = np.array(Y)
    print('X:'+ str(X.shape),'Y:'+str(Y.shape),'legal_encoded_seq: ' + str(len(legal_encoded_seq)))
    return X,Y,legal_encoded_seq

In [ ]:
def split_by_id(df,label):
    positive_df = df[df['legal_encoded'] == label]
    negative_df = df[~df.case_id.isin(positive_df.case_id)]
    positive_df['label'] = 1
    negative_df['label'] = 0
    x2 = df[df['legal_encoded'] == label].iloc[0].legal_content_token
    label_df = pd.concat([positive_df, negative_df], ignore_index=True, sort=False)
    
    train_indx, rest_indx = next(sklearn.model_selection.GroupShuffleSplit(test_size=.30, n_splits=2, random_state = 42).split(label_df, groups= label_df['case_id']))
    rest_df = label_df.iloc[rest_indx]
    dev_indx,test_indx = next(sklearn.model_selection.GroupShuffleSplit(test_size=.50, n_splits=2, random_state = 42).split(rest_df, groups=rest_df['case_id']))

    train = label_df.iloc[train_indx]
    dev = rest_df.iloc[dev_indx]
    test = rest_df.iloc[test_indx]
    
    def get_data_by_id(df,x2):
        X = [] 
        Y = []
    
        for i,row in df.iterrows():
            x = row['plaintiff_token']
            X.append([x, x2])
            Y.append(row['label'])
        X = np.array(X ,dtype=object)
        Y = np.array(Y)
        print('X:'+ str(X.shape),'Y:'+str(Y.shape))
        return X,Y
    
    X_train,Y_train = get_data_by_id(train,x2)
    X_dev,Y_dev = get_data_by_id(dev,x2)
    X_test,Y_test = get_data_by_id(test,x2)
    
    return X_train,Y_train,X_dev,Y_dev,X_test,Y_test

In [ ]:
from tqdm import tqdm_notebook as tqdm
def featurized(array1,array2,feature_list = ['word_overlap' , 'word_cross_product' , 'wordnet_feature_po', 'wordnet_feature_per' ]):
    #word_overlap , word_cross_product , wordnet_feature [hyponyms , hypernyms] 
    """Ex : feature_list = ['word_overlap' , 'word_cross_product' , 'wordnet_feature_po', 'wordnet_feature_per' ]"""
    feature_dict_seq = []
    for s1,s2 in tqdm(zip(array1,array2)): 
        feature_dict = {}
        
    #word_overlap
        if 'word_overlap' in feature_list: 
            feature_dict.update(dict(word_overlap(s1,s2)))

    #word_cross_product + wordnet_feature [hyponyms , hypernyms] 
        if 'word_cross_product' in feature_list:
            feature_dict.update(dict(word_cross_product(s1,s2,feature_list)))
        
        feature_dict_seq.append(feature_dict)

    return feature_dict_seq

# Evaluation functions

In [ ]:
def recall(y_pred,y_true,legal_encoded_seq):
    
    new_pred = []
    new_true = []
    TP_dict = Counter()
    FN_dict = Counter()
    TP = 0
    FN = 0
    all_score = {key : 0 for key in legal_encoded_seq}
    
    for i in range(len(y_pred)):
        pred = y_pred[i]
        true = y_true[i]
        legal_encoded = legal_encoded_seq[i]
        
        if pred == 1:
            new_pred.append(legal_encoded)
        else:
            new_pred.append(0)
        
        if true == 1:
            new_true.append(legal_encoded)
        else:
            new_true.append(0)
            
    for label1,label2 in zip(new_pred,new_true):
        if label2 != 0 and label1 == label2:
            TP_dict[label2] += 1
        elif label2 != 0 and label1 != label2:
            FN_dict[label2] += 1
    
     
    for legal_encoded in legal_encoded_seq:
        TP = TP_dict[legal_encoded]
        FN = FN_dict[legal_encoded]
        recall = TP / (TP+FN)
        all_score[legal_encoded] = recall
        
    print('recall :')
    return all_score

In [ ]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def microf1(y_true, y_pred):

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


def macrof1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def evaluate(y_true,y_pred,legal_encoded_seq) :
    all_legal_dict = {key : {'true' : [], 'pred' : []} for key in legal_encoded_seq}
    all_score_dict = {key : {} for key in legal_encoded_seq}
    
    for i in range(len(legal_encoded_seq)) :
        true = y_true[i]
        pred = y_pred[i]
        legal_encoded = legal_encoded_seq[i]
        all_legal_dict[legal_encoded]['true'].append(true)
        all_legal_dict[legal_encoded]['pred'].append(pred)
        
    for key in all_legal_dict:
        true = all_legal_dict[key]['true']
        pred = all_legal_dict[key]['pred']
        f1 =  f1_score(true,pred,labels=[1])
        precision = precision_score(true,pred,labels=[1])
        recall = recall_score(true,pred,labels=[1])
        all_score_dict[key]['f1'] = f1
        all_score_dict[key]['precision'] = precision
        all_score_dict[key]['recall'] = recall

    return all_score_dict



# Train Logistic regression

In [ ]:
label = 100

In [ ]:
vectorizer = DictVectorizer(sparse=True)

X_train,Y_train,X_dev,Y_dev,X_test,Y_test = split_by_id(df,label)
X_train_featurized = featurized(list(X_train[:, 0]),list(X_train[:, 1]))
X_train = vectorizer.fit_transform(X_train_featurized)

model = LogisticRegression(solver='liblinear', n_jobs=1, verbose=1, random_state=42, C=1e5, penalty='l2', max_iter=2000)
model.fit(X_train, Y_train)

X_test_featurized = featurized(list(X_test[:, 0]),list(X_test[:, 1]))
X_test = vectorizer.transform(X_test_featurized)
    
print("Evaluation")    

y_pred = model.predict(X_test)    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


X:(9932, 2) Y:(9932,)
X:(2202, 2) Y:(2202,)
X:(2298, 2) Y:(2298,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



[LibLinear]


Evaluation


In [ ]:
print(label)
print("Evaluation")

print('f1 : ')
print(f1_score(Y_test, y_pred,labels=[1]))  
print('------------------------------------------------------------ ')   
print('precision : ')
print(precision_score(Y_test, y_pred,labels=[1]))   
print('------------------------------------------------------------ ')
print('recall : ')
print(recall_score(Y_test, y_pred,labels=[1]))
print('------------------------------------------------------------ ')
print(classification_report(Y_test, y_pred))   

100
Evaluation
f1 : 
0.0
------------------------------------------------------------ 
precision : 
0.0
------------------------------------------------------------ 
recall : 
0.0
------------------------------------------------------------ 
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      2280
           1       0.00      0.00      0.00        18

    accuracy                           0.94      2298
   macro avg       0.50      0.48      0.49      2298
weighted avg       0.98      0.94      0.96      2298

